# 1.1 Introducción

## Creando el ambiente de trabajo

## Con Python

Con python instalado en nuestro computador, vamos a crear un ambiente virtual con el siguiente comando:

* python -m venv env (Windows/Unix/macOS)

Para activar este entorno virtual, se ejecutara el siguiente comando:

* .\env\Scripts\activate (Windows)
* source env/bin/activate (Unix/macOS)

Para desactivar el entorno virtual, ejecute el siguiente comando:

* deactivate

Mas información sobre los ambientes virtuales nativos de python en el siguiente [link](https://packaging.python.org/en/latest/guides/installing-using-pip-and-virtual-environments/#using-requirements-files).

## Con Codespaces de Github

Para crear un entorno virtual, se ejecutara el siguiente comando:

* virtualenv ~/.venv

Para activar este entorno virtual, se ejecutara el siguiente comando:

* source ~/.venv/bin/activate

## Con Anaconda

Todos los comandos de conda se deben ejecutar en el terminal en donde tengan instalado Conda. En windows puede ejecutarse en el terminal si es que se agrega el path a las variables de entorno, la instalación por defecto no se agrega y se instala el terminal de anaconda (anaconda prompt). En macOS solo ejecuta en el terminal nativo.

Crear un ambiente virtual de anaconda con el siguiente comando:

* conda create --name env python=3.8

Para activar este entorno virtual, se ejecutara el siguiente comando: 

* conda activate env 

Para desactivar este ambiente, se ejecutara el siguiente comando:

* conda deactivate

Para eliminar el ambiente 

* conda remove --name env --all

# Preprocessing

In [1]:
import datasets # Biblioteca de manejo de conjuntos de datos para procesamiento de lenguaje natural
import es_core_news_sm # Modelo Spacy de procesamiento de lenguaje natural en español
import spacy # Biblioteca de procesamiento de lenguaje natural
import pandas as pd # Biblioteca de manejo de conjuntos de datos
import re # Módulo de expresiones regulares
import tokenizers # Biblioteca de tokenización de texto
import nltk # Biblioteca de procesamiento de lenguaje natural
from pathlib import Path # Biblioteca para manejo de paths relativos
import tqdm # Biblioteca que proporciona una barra de progreso para bucles y tareas iterativas en Python
import os # Módulo incorporado en Python que proporciona funciones para interactuar con el sistema operativo
import csv # Módulo incorporado en Python que proporciona funciones para leer y escribir archivos CSV

2023-06-01 01:01:41.074403: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-01 01:01:43.518572: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-01 01:01:43.522514: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 01:01:48.037093: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#path relativo
data_path=Path('..')

## Importar csv o txt

Encoding

* UTF-8: 'utf-8' es el encoding predeterminado en Pandas y es ampliamente utilizado para archivos CSV. Es compatible con una amplia gama de caracteres y es recomendado si no estás seguro del encoding del archivo.

* Latin-1 (ISO 8859-1): 'latin-1' es otro encoding común utilizado en archivos CSV generados por aplicaciones en entornos Windows. Es compatible con una gran cantidad de idiomas europeos.

* UTF-16: 'utf-16' es un encoding de Unicode que utiliza 16 bits para representar los caracteres. Es útil cuando se trabaja con idiomas que tienen una gran cantidad de caracteres, como algunos idiomas asiáticos.

* UTF-32: 'utf-32' es un encoding de Unicode que utiliza 32 bits para representar los caracteres. Al igual que UTF-16, es útil para idiomas con una gran cantidad de caracteres.

* ASCII: 'ascii' es un encoding básico que solo admite caracteres en inglés y no es compatible con caracteres acentuados u otros caracteres especiales.

### Con el comando `open()`

In [20]:
i=0
with open(data_path/'spanish_diagnostics/spanish_diagnostics.csv', 'r') as file:
  csvreader = csv.reader(file, delimiter=',')
  for row in csvreader:
    print(row)
    i+=1
    if i==5:
      break

['diagnostic', 'is_dental']
['- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR', '1']
['OBTRUCCION FOSA NASAL DERECHA', '0']
['Perturbación de la actividad y de la atención Trastorno defícit atencional', '0']
['M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 ALGIA PELVICA HTA CRONICA', '0']


In [21]:
i=0
with open(data_path/'spanish_diagnostics/spanish_diagnostics.csv', 'r',encoding='latin-1') as file:
  csvreader = csv.reader(file, delimiter=',')
  for row in csvreader:
    print(row)
    i+=1
    if i==5:
      break

['diagnostic', 'is_dental']
['- ANOMALÃ\x8dAS DENTOFACIALES (INCLUSO LA MALOCLUSIÃ\x93N)\n\n\n DISCREPANCIA DENTOMAXILAR', '1']
['OBTRUCCION FOSA NASAL DERECHA', '0']
['PerturbaciÃ³n de la actividad y de la atenciÃ³n Trastorno defÃ\xadcit atencional', '0']
['M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 ALGIA PELVICA HTA CRONICA', '0']


### Con los dataframes de `Pandas`

In [24]:
spanish_diagnostics = pd.read_csv(data_path/'spanish_diagnostics/spanish_diagnostics.csv')

In [25]:
spanish_diagnostics[0:5]

,diagnostic,is_dental
0,- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUS...,1
1,OBTRUCCION FOSA NASAL DERECHA,0
2,Perturbación de la actividad y de la atención ...,0
3,M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 A...,0
4,PIEZA 3 CARIES DENTINARIA PROFUNDA PROXIMA A C...,1


In [26]:
spanish_diagnostics_malo = pd.read_csv(data_path/'spanish_diagnostics/spanish_diagnostics.csv',encoding='latin-1')

In [27]:
spanish_diagnostics_malo[0:5]

,diagnostic,is_dental
0,- ANOMALÃAS DENTOFACIALES (INCLUSO LA MALOCLU...,1
1,OBTRUCCION FOSA NASAL DERECHA,0
2,PerturbaciÃ³n de la actividad y de la atenciÃ³...,0
3,M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 A...,0
4,PIEZA 3 CARIES DENTINARIA PROFUNDA PROXIMA A C...,1


### ¿Que pasa si queremos importar muchos archivos txt? 

In [99]:
#funcion de ejemplo para crear archivos a modo de ejemplo con el dataset anterior. No es importante para este laboratorio.
def csv2txt(ruta_csv,ruta_carpeta,cantidad):
    i=0
    if ruta_carpeta.exists():
        pass
    else:
        ruta_carpeta.mkdir(parents=True, exist_ok=True)

    with open(ruta_csv, 'r') as file:
        csvreader = csv.reader(file, delimiter=',')
        headers=next(csvreader)
        for index,row in enumerate(csvreader):
            filename = f'archivo_{index}.txt'
            with open(ruta_carpeta/filename, 'w') as file_txt:
                file_txt.write(row[0])
            
            i+=1

            if i==cantidad:
                break
            

path_csv=data_path/'spanish_diagnostics/spanish_diagnostics.csv'
path_carpeta=data_path/'data'/'ejemplo'

csv2txt(path_csv,path_carpeta,5)

### una manera de leerlos

In [73]:
archivos=list(path_carpeta.glob('*.txt')) #que archivos existen en la carpeta
archivos_ordenados=sorted(archivos, key=os.path.getmtime) #aca solo se ordena los archivos por fecha de creación
[ a.name for a in archivos_ordenados ] #aca solo se presentan los nombres

['archivo_0.txt',
 'archivo_2.txt',
 'archivo_4.txt',
 'archivo_1.txt',
 'archivo_3.txt']

In [74]:
for archivo in archivos_ordenados:
    with open(archivo, 'r') as file:
            lines = file.readlines()
            print(lines)

['- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n', '\n', '\n', ' DISCREPANCIA DENTOMAXILAR']
['Perturbación de la actividad y de la atención Trastorno defícit atencional']
['PIEZA 3 CARIES DENTINARIA PROFUNDA PROXIMA A CAMARA PULPAR, EVALUAR POR ESPECIALIDAD']
['OBTRUCCION FOSA NASAL DERECHA']
['M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 ALGIA PELVICA HTA CRONICA']


## 🤗 Datasets

🤗 (HuggingFace) Datasets es una biblioteca de manejo de conjuntos de datos para procesamiento de lenguaje natural que se destaca por la simplicidad de sus métodos y el gran repositorio 🤗 Hub que contiene muchos conjuntos de datos libres para descargar sólo con una linea de Python.

En nuestro curso trabajaremos con `spanish_diagnostics`, un conjunto de datos de nuestro grupo investigación PLN@CMM que contiene textos de sospechas diagnósticas de la lista de espera chilena y está etiquetado con el destino de la interconsulta; este destino puede ser `dental` o `no_dental`.

In [75]:
spanish_diagnostics = datasets.load_dataset('fvillena/spanish_diagnostics') # Con esta linea descargamos el conjunto de datos completo

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset spanish_diagnostics downloaded and prepared to /home/vscode/.cache/huggingface/datasets/fvillena___spanish_diagnostics/default/0.0.0/45c176cea64580ea9631f78c2867a657ede368597681e5337e9f1c976e4e84ff. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Nuestro conjunto de datos cuenta con 2 particiones, una partición `train` y otra `test`.

In [76]:
spanish_diagnostics

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 30000
    })
})

En esta clase utilizaremos la partición `train` del conjunto de datos.

In [77]:
spanish_diagnostics["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 70000
})

Podemos acceder facilmente a atributos de nuestro `Dataset`.

- `shape`: Tal como en muchas otras bibliotecas de python este atributo contiene la forma de nuestro conjunto de datos con la sintaxis `(filas, columnas)`.
- `column_names`: Este atributo contiene el nombre de las características que tiene nuestro conjunto de datos. En nuestro caso tenemos una característica `text`, la cual contiene la hipótesis diagnóstica del conjunto de datos y `label` que contiene el destino al cual fue referido.
- `features`: Este atributo nos describe la clase a la que pertenece cada una de las características. En nuestro caso `text` es un `string` y `label` es del tipo `ClassLabel` con 2 clases con nombre `not_dental` y `dental`.

In [78]:
spanish_diagnostics["train"].shape

(70000, 2)

In [79]:
spanish_diagnostics["train"].column_names

['text', 'label']

In [80]:
spanish_diagnostics["train"].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_dental', 'dental'], id=None)}

Tal como en muchas otras clases de datos en Python podemos acceder a subconjuntos de datos a través de sus índices.

In [81]:
spanish_diagnostics["train"][0]

{'text': '- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
 'label': 1}

In [82]:
spanish_diagnostics["train"][:3]

{'text': ['- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
  'OBTRUCCION FOSA NASAL DERECHA',
  'Perturbación de la actividad y de la atención Trastorno defícit atencional'],
 'label': [1, 0, 0]}

In [83]:
spanish_diagnostics["train"][1,3,5]

{'text': ['OBTRUCCION FOSA NASAL DERECHA',
  'M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 ALGIA PELVICA HTA CRONICA',
  'pieza n 3.4 tratada endodonticamente, restaurada con ionomero y resina compuesta. Necesita protesis fija por gran pNrdida coronaria'],
 'label': [0, 0, 1]}

También podemos acceder a cada una de las características por separado.

In [84]:
spanish_diagnostics["train"]['text'][:3]

['- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
 'OBTRUCCION FOSA NASAL DERECHA',
 'Perturbación de la actividad y de la atención Trastorno defícit atencional']

Con 🤗 Datasets también podemos trabajar en otras bibliotecas, como por ejemplo importar el conjunto de datos en Pandas.

In [85]:
spanish_diagnostics_train_df = pd.DataFrame(spanish_diagnostics["train"])

In [86]:
spanish_diagnostics_train_df

,text,label
0,- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUS...,1
1,OBTRUCCION FOSA NASAL DERECHA,0
2,Perturbación de la actividad y de la atención ...,0
3,M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 A...,0
4,PIEZA 3 CARIES DENTINARIA PROFUNDA PROXIMA A C...,1
...,...,...
69995,DM1 Evaluación,1
69996,ABCESO SUBMUCOSO PIEZA 2.6,1
69997,Pbs Inmunodeficiencia,0
69998,"QUISTE SINOVIAL DEL HUECO POPLITEO, DE BAKER",0


Verificamos que nuestro conjunto de datos tiene sus clases balanceadas.

In [87]:
spanish_diagnostics_train_df.label.value_counts()

label
1    35034
0    34966
Name: count, dtype: int64

Si tenemos localmente un conjunto de datos y queremos importarlo a 🤗 Datasets también podemos hacerlo. Aquí importamos el conjunto de datos desde un archivo CSV.

In [90]:
path_datos=str(data_path/'spanish_diagnostics/spanish_diagnostics.csv')
datasets.load_dataset('csv', data_files=path_datos)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/vscode/.cache/huggingface/datasets/csv/default-dfd19ccd89d95031/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['diagnostic', 'is_dental'],
        num_rows: 100000
    })
})

## Normalización

Una de las tareas que podemos realizar sobre las características no estructuradas de texto es la normalización. La cual consiste en llevar nuestro texto a una forma más consistente a lo largo del conjunto de datos.

Podemos observar que nuestro conjunto de datos cuenta con una alta inconsistencia respecto al uso de mayúsculas, el uso de tildes y el uso de signos de puntuación.

In [91]:
spanish_diagnostics["train"]["text"][:10]

['- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
 'OBTRUCCION FOSA NASAL DERECHA',
 'Perturbación de la actividad y de la atención Trastorno defícit atencional',
 'M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 ALGIA PELVICA HTA CRONICA',
 'PIEZA 3 CARIES DENTINARIA PROFUNDA PROXIMA A CAMARA PULPAR, EVALUAR POR ESPECIALIDAD',
 'pieza n 3.4 tratada endodonticamente, restaurada con ionomero y resina compuesta. Necesita protesis fija por gran pNrdida coronaria',
 'PZ. 12 TREPANADA',
 'CARCINOMA TORIODEO',
 'DISPEPSIA Y METEORISMO',
 'ASA 1 DENTICION TEMPORAL MORDIDA CRUZADA']

Para poder llevar todo a minúsculas, simplemente podemos utilizar el método str.lower().

In [92]:
sample_sentence_lower = spanish_diagnostics["train"]["text"][0].lower()
sample_sentence_lower

'- anomalías dentofaciales (incluso la maloclusión)\n\n\n discrepancia dentomaxilar'

### Expresiones regulares

Las expresiones regulares son una herramienta poderosa para manipular y buscar patrones en cadenas de texto. Las expresiones regulares en Python se definen como una secuencia de caracteres que especifican un patrón de búsqueda. Puedes utilizarlas para realizar tareas como validar formatos de cadenas, extraer información específica de un texto o reemplazar partes de una cadena. En python tenemos el paquete re, sus principales funciones son las siguientes:


* re.search(pattern, string): Busca el patrón en toda la cadena y devuelve un objeto "Match" si encuentra una coincidencia. Puedes utilizar métodos como group() para obtener la cadena que coincide con el patrón.

* re.match(pattern, string): Busca el patrón solo al comienzo de la cadena y devuelve un objeto "Match" si encuentra una coincidencia.

* re.findall(pattern, string): Busca todas las coincidencias del patrón en la cadena y devuelve una lista de cadenas que cumplen con el patrón.

* re.sub(pattern, repl, string): Busca todas las coincidencias del patrón en la cadena y las reemplaza con la cadena de reemplazo especificada.

Para definir un patrón de expresión regular, puedes utilizar varios caracteres especiales y secuencias de escape. Algunos de los caracteres especiales comunes incluyen:

* `.` : Coincide con cualquier carácter excepto una nueva línea.
* * Ejemplo: a.b coincide con "aab", "a1b", "a@b", etc., pero no con "a\nb". 
* `*` : Coincide con cero o más repeticiones del elemento anterior.
* * Ejemplo: ab*c coincide con "ac", "abc", "abbc", "abbbc", etc.
* `+` : Coincide con una o más repeticiones del elemento anterior.
* * Ejemplo: ab+c coincide con "abc", "abbc", "abbbc", etc., pero no con "ac".
* `^` : Coincide con el inicio de una cadena o línea.
* * Ejemplo: ^Start coincide con "Start of line", pero no con "End of line: Start".
* `$` : Coincide con el final de una cadena o línea.
* * Ejemplo: end$ coincide con "End of line", pero no con "Start of line: End".
* `?` : Coincide con cero o una repetición del elemento anterior.
* * Ejemplo: colou?r coincide con "color" y "colour".
* `[ ]`: Coincide con cualquier carácter dentro de los corchetes.
* * Ejemplo: `[aeiou]` coincide con cualquier vocal en minúscula.
* `( )` : Agrupación de elementos y captura de grupos.
* * Ejemplo: (ab)+ coincide con "ab", "abab", "ababab", etc.
* `\` : Se utiliza como carácter de escape para caracteres especiales o para dar significado especial a ciertos caracteres.
* * Ejemplo: \d coincide con cualquier dígito, \b coincide con una posición en la cadena donde hay un cambio de caracteres de palabra a no palabra o viceversa.
* `|` : Coincide con uno de los patrones separados por el operador "|".
* * Ejemplo: cat|dog coincide con "cat" o "dog".


Para eliminar todo los caracteres no alfabéticos podemos utilizar un patrón de expresión regular con la sintaxis: `[^a-zñáéíóú]`, la cual se explica como:

- `^`: Este es un `NO` lógico que invierte todo lo que viene a su derecha.
- `a-z`: Este patrón coincide todos los caracteres de la `a` a la `z` (minúsculas)
- `áéíóú`: Este patrón coincide con todas las vocales con tilde.

Todos estos patrones están concatenados con un `O` lógico.

In [94]:
sample_sentence_lower_alpha = re.sub(r'[^a-zñáéíóú]', ' ', sample_sentence_lower)
sample_sentence_lower_alpha

'  anomalías dentofaciales  incluso la maloclusión     discrepancia dentomaxilar'

Reemplazamos todas las vocales con tilde con con su forma sin tilde.

In [95]:
re.sub('ó', 'o', sample_sentence_lower_alpha)

'  anomalías dentofaciales  incluso la maloclusion     discrepancia dentomaxilar'

Agrupamos todo en una función que normalizará una cadena de texto que le pasemos.

In [96]:
def normalize(text, remove_tildes = True):
    """Normaliza una cadena de texto convirtiéndo todo a minúsculas, quitando los caracteres no alfabéticos y los tildes"""
    text = text.lower() # Llevamos todo a minúscula
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) # Reemplazamos los caracteres no alfabéticos por un espacio
    if remove_tildes:
        text = re.sub('á', 'a', text) # Reemplazamos los tildes
        text = re.sub('é', 'e', text)
        text = re.sub('í', 'i', text)
        text = re.sub('ó', 'o', text)
        text = re.sub('ú', 'u', text)
    return text

Los objetos del tipo Dataset implementan un método Dataset.map() con el cual podemo aplicar uan función a cada una de las instancias de nuesto conjunto de datos. Lo interesante de este método es que aplica la función de manera paralela.

In [97]:
spanish_diagnostics_normalized = spanish_diagnostics["train"].map(
    lambda x: { # Utilizamos una función anónima que devuelve un diccionario
        "normalized_text" : normalize(x["text"]) # Esta es una nueva característica que agregaremos a nuestro conjunto de datos.
    })

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

Ahora nuestro conjunto de datos cuenta con una nueva característica `normalized_text`.

In [98]:
spanish_diagnostics_normalized[0]

{'text': '- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
 'label': 1,
 'normalized_text': '  anomalias dentofaciales  incluso la maloclusion     discrepancia dentomaxilar'}